# **Файлы и файловая система**

Как правило, при анализе данных для чтения файла с диска и загрузки данных из него используеются такие высокоуровневые средства, как `pandas.read_csv`. Однако важно понимать основы работы с файлами в Python.

Чтобы открыть файл для чтения и записи, следует использовать встроенную функцию `open`, которая принимает относительный или абсолютный путь до файла:

In [ ]:
path = 'ch13/segismundo.txt' #описываем путь до файла и записываем его в переменную path
f = open(path) #открываем path на чтение (режим r) и записываем его в переменную f

По умолчанию файл открывается только для чтения — в режиме `r`. Далее описатель файла `f` можно рассматривать как список и перебирать строки:

In [ ]:
for line in f: #для каждой строки списка (переменная f, описатель файла, является список из строк)
  pass #код для перебора строк

У строк, прочитанных из файла, сохраняется признак конца строки (EOL), поэтому часто можно встретить код, который удаляет концы строк:

In [ ]:
lines = [x.rstrip() for x in open(path)] #построчно читаем файл и открываем его как список из строк

Если для создания объекта файла использовалась функция `open`, то следует явно закрывать файл по завершении работы с ним. Закрытие файла возвращает ресурсы операционной системе:

In [ ]:
f.close() #закрываем файл

Упростить эту процедуру позволяет предложение `with`:

In [ ]:
with open(path) as f: #открывает файл как f и закрывает его в конце кода
  lines = [x.rstrip() for x in f] #построчно читаем файл и открываем его как список из строк

В таком случае файл `f` автоматически закрывается при выходе из блока `with`. Если бы мы написали `f = open(path, 'w')`, то был бы создан *новый* файл *examples/segismundo.txt*, а старый был бы перезаписан.

Существует также режим `x`, в котором создается допускающий запись файл, но лишь в том случае, если его еще не существует, в противном случае возбуждается исключение. Допустимые режимы ввода-вывода:

In [ ]:
r #режим чтения
w #режим записи. Создается новый файл (старый с тем же именем удаляется)
a #дописывание в конец существующего файла (если файла нет, его создают)
r+ #чтение и запись
b #уточнение режима для двоичных файлов: 'rb' или 'wb'
t #текстовый режим (байты автоматически декодируются в Unicode). Этот режим подразумевается по умолчанию, если не указано противное. Букву t можно комбинировать с другими режимами (например, 'rt' или 'xt')

При работе с файлами, допускающими чтение, чаще всего употребляются методы `read`, `seek` и `tell`. Метод `read` возвращает определенное число символов из файла. "Символ" определяется кодировкой файла. Если же файл открыт в двоичном режиме, то под символами понимаются просто байты:

In [ ]:
f = open(path)
f.read(10) #читает 10 символов из файла
f2 = open(path, 'rb') #открывает файл в двоичном режиме
f2.read(10) #читает 10 байтов из файла

Метод `read` продвигает указатель файла вперед на количество прочитанных байтов. Метод `tell` сообщает текущую позицию:

In [ ]:
f.tell() #11
f2.tell() #10

Хотя мы прочитали из файла 10 символов, позиция равна 11, потому что именно столько байтов пришлось прочитать, чтобы декодировать 19 символов в подразумеваемой по умолчанию кодировке файла. Чтобы узнать кодировку по умолчанию, используется модуль `sys`:

In [ ]:
import sys #импортируем модуль
sys.getdefaultencoding() #программа, описывающая кодировку файла по умолчанию

Метод `seek` изменяет позицию в файле на указанную:

In [ ]:
f.seek(3) #3

Для записи текста в файл служат методы `write` или `writelines`. Например, можно было бы создать вариант файла без пустых строк:

In [ ]:
with open('tmp.txt', 'w') as handle: #создаем новый файл в режиме записи
  handle.writelines(x for x in open(path) if len(x)>1) #записываем в файл все строки, длина которых больше 1
with open('tmp.txt') as f: #открываем файл, который создали в предыдущем 
  lines = f.readlines() #записывает в переменную lines список прочитанных из файла строк

Наиболее употребительные методы и атрибуты для работы с файлами в Python:

In [ ]:
read([size]) #возвращает прочитанные из файла данные в виде строки. Необязательный аргумент size сообщает, сколько байтов читать
readlines([size]) #возвращает список прочитанных из файла строк. Необязательный аргумент size сообщает, сколько строк читать
write(str) #записывает переданную строку в файл
writelines(strings) #записывает  переданную последовательность строк в файл
close() #закрывает описатель файла
flush() #сбрасывает внутренний буфер ввода-вывода на диск
seek(pos) #перемещает указатель чтения-записи на байт файла с указанным номером
tell() #возвращает текущую позицию в файле в виде целого числа
closed #True, если файл закрыт

# **Байты и Unicode в применении к файлам**

По умолчанию Python открывает файлы (как для чтения, так и для записи) в *текстовом режиме*, предполагая, что вы намереваетесь работать со строками (которые хранятся в Unicode). Чтобы открыть файл в *двоичном режиме*, следует добавить к основному режиму букву `b`:

In [ ]:
with open(path) as f:
  chars = f.read(10) #прочитает 10 символов

UTF-8 — это кодировка Unicode переменной длины, поэтому Python читает столько байтов, чтобы после декодирования получилось указанное количество символов (10), их может быть всего 10, а может быть и целых 40. Если вместо этого октрыть файл в режиме `'rb'`, то `read` прочитает ровно столько байтов, сколько запрошено:

In [ ]:
with open(path, 'rb') as f:
  data = f.read(10) #прочитает 10 байтов

Зная кодировку текста, можно декодировать байты в объект `str` самостоятельно, но только в том случае, если последовательность байтов корректна и полна:

In [ ]:
data.decode('utf8') #декодирует 10 байтов в кодировке UTF-8

Текстовый режим в сочетании с параметром `encoding` функции `open` — удобный способ преобразовать данные из одной кодировки Unicode в другую:

In [ ]:
sink_path = 'sink.txt'
with open(path) as source:
  with open(sink_path, 'xt', encoding='iso–8859–1') as sink:
    sink.write(source.read())
with open(sink_path, encoding='iso–8859–1') as f:
  print(f.read(10)) #

Следует быть осторожным, вызывая метод `seek` для файла, открытого не в  двоичном режиме. Если указанная позиция окажется в  середине последовательности байтов, образующих один символ Unicode, то последующие операции чтения завершатся ошибкой:

In [ ]:
f = open(path)
f.read(5) #'Sueńa'
f.seek(4) #4
f.read(1) #UnicodeDecodeError

Чтобы открывать файлы в `google colab` из `google drive` применяется мледующий код


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
with open('/content/drive/MyDrive/file.txt') as f: #открывает файл как f и закрывает его в конце кода
  lines = [x.rstrip() for x in f] #построчно читаем файл и открываем его как список из строк

In [16]:
a=[]
for element in lines:
  a.append(rtuple(strip(element)))
print(a)

[['1', '2', '3', ',', ' ', '4', '5', '6'], ['1', '5', '6', ',', ' ', '6', '6', '6'], ['1', '2', '3', ',', ' ', '3', '4', '5'], ['1', '5', '6', ',', ' ', '5', '6', '1']]
